In [1]:
#%reload_ext tensorboard
import numpy as np
import json
import tensorflow as tf
import os

import time
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import gym

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=512)])
  except RuntimeError as e:
    print(e)
    
    
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_logical_devices('GPU')

import tf_agents
from tf_agents.networks import categorical_q_network
from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.environments import py_environment, parallel_py_environment, batched_py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, element_wise_squared_loss
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.policies import policy_saver
import logging

import tensorflow.keras as keras

tf.compat.v1.enable_v2_behavior()
import time
import json
import datetime
import copy
import shutil

#import imp
from scan_gym import envs
from scan_gym.envs.ScannerEnv2 import space_carving
#imp.reload(envs)
import csv

seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

import policy_test

GPUs:  1


In [2]:
def calculate_position(init_state,steps):
    n_positions = 180
    n_pos = init_state + steps
    if n_pos>(n_positions-1):
        n_pos -= n_positions
    elif n_pos<0:
        n_pos += n_positions
    return n_pos


def run_rnd_episode_test(env,n_images):
    state = env.reset()
    actions = []
    images = []
    rewards = []
    images.append([env.current_theta,env.current_phi])
    for i in range(1,n_images):
        action = np.random.randint(env.nA)
        actions.append(action)
        time_step = env.step(action)
        images.append([env.current_theta,env.current_phi])
        rewards.append(time_step.reward)
        if time_step.step_type == 2:
            break
    return actions, images, env.theta_bias, rewards, env.total_reward , env.spc.gt_compare_solid(), env.spc.volume



def run_episode_test(env,tf_env,policy):
    state = tf_env.reset()
    time_steps = 90
    actions = []
    images = []
    rewards = []
    images.append([env.current_theta,env.current_phi])
    for i in range(1,time_steps):
        action_step = policy.action(state)
        actions.append(int(action_step.action.numpy()[0]))
        state = tf_env.step(action_step.action)
        
        images.append([env.current_theta,env.current_phi])
        rewards.append(float(state.reward.numpy()[0]))
        if state.is_last():
            break
    return actions, images, env.theta_bias, rewards, env.total_reward , env.spc.gt_compare_solid(), env.spc.volume



def run_uniform_test(models_path,model,n_images, init_theta):
    total_theta_positions = 180
    images = []
    spc = space_carving.space_carving_rotation_2d( os.path.join(models_path, model[0]),
                        gt_mode=True, theta_bias=0,
                        total_theta_positions=total_theta_positions,
                        cube_view='static')
    
    dist = total_theta_positions//n_images

    theta_count = init_theta
    for i in range(n_images):
        spc.carve(theta_count,0)
        images.append([theta_count,0])
        theta_count = calculate_position(theta_count,dist)
        
    gt_sim = spc.gt_compare_solid()
    return images, gt_sim, spc.volume



def run_episode_test(env,tf_env,policy):
    state = tf_env.reset()
    time_steps = 90
    actions = []
    images = []
    rewards = []
    images.append([env.current_theta,env.current_phi])
    for i in range(1,time_steps):
        action_step = policy.action(state)
        actions.append(int(action_step.action.numpy()[0]))
        state = tf_env.step(action_step.action)
        
        images.append([env.current_theta,env.current_phi])
        rewards.append(float(state.reward.numpy()[0]))
        if state.is_last():
            break
    return actions, images, env.theta_bias, rewards, env.total_reward , env.spc.gt_compare_solid(), env.spc.volume

In [3]:
def plot_vol_cam(vol,cam_pts):
    # for creating a responsive plot
    %matplotlib widget
    # importing required libraries
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm

    def plot_colourline(x,y,z,c,ax):
        c = cm.jet((c-np.min(c))/(np.max(c)-np.min(c)))
        for i in np.arange(len(x)-1):
            ax.plot([x[i],x[i+1]], [y[i],y[i+1]],[z[i],z[i+1]], c=c[i])
        return
    
    def minMaxNorm(old, oldmin, oldmax , newmin , newmax):
        return ( (old-oldmin)*(newmax-newmin)/(oldmax-oldmin) ) + newmin
    
    
    def a2c(angles, R=5,n_theta=180,n_phi=4):
        theta= angles[:,0]*2*np.pi/n_theta
        phi=angles[:,1]*.5*np.pi/n_phi
        x = R *np.cos(phi) * np.cos(theta) #x pos of camera
        y = R *np.cos(phi) * np.sin(theta) #y pos of camera   
        z = R *np.sin(phi)
        return [x,y,z]
    
    x,y,z = np.where(vol==1)
    x = minMaxNorm(x,0,63,-4,4)
    y = minMaxNorm(y,0,63,-4,4)
    z = minMaxNorm(z,0,63,-4,4)

    # creating figure
    fig = plt.figure(figsize=(10,8))
    ax = Axes3D(fig)

    ax.set_xlim3d(-5.5, 5.5)
    ax.set_ylim3d(-5.5, 5.5)
    ax.set_zlim3d(-5.5, 5.5)

    # creating the plot
    p = ax.scatter(x, y, z, color='green',s=1)

    #x,y,z = np.where(spc.sc.values()==0)
    cam_pts = np.array(cam_pts)
    crd = a2c(cam_pts)
    ax.scatter(crd[0], crd[1], crd[2], color='red',s=20)
    plot_colourline(crd[0], crd[1], crd[2], np.arange(len(crd[0])), ax)
    fig.colorbar(p)

    # setting title and labels
    ax.set_title("3D plot")
    ax.set_xlabel('x-axis')
    ax.set_ylabel('y-axis')
    ax.set_zlabel('z-axis')
    ax.grid()
    
    #make  0 origin coincide in all axis
    ax.xaxis._axinfo['juggled'] = (0,0,0)
    ax.yaxis._axinfo['juggled'] = (1,1,1)
    ax.zaxis._axinfo['juggled'] = (2,2,2)
    
    ax.view_init(90, 0)

    # displaying the plot
    plt.show()

In [4]:
policy_name ='218_20220613-112522'
policies_path = '/home/pico/uni/romi/policies_test'
policy = tf.compat.v2.saved_model.load(os.path.join(policies_path,policy_name))

In [5]:
models_path  = '/home/pico/uni/romi/scanner-gym_models_v2'
models = ['207_2d','208_2d','209_2d','210_2d','211_2d']
'''train_models = ['207_2d','208_2d','209_2d', '210_2d',
               '211_2d','212_2d','213_2d' ,'214_2d']'''
train_models = ['218_2d']
n_images = 10
continuous = False

#scan_env = gym.make('ScannerEnv-v1', models_path=models_path, train_models=models,
#                   n_images = n_images, continuous=continuous, gt_mode=True, cube_view='static')

env = suite_gym.load('ScannerEnv-v2',gym_kwargs={'models_path':models_path, 'train_models':train_models,
                                                   'n_images':n_images, 'continuous':continuous,
                                                   'gt_mode':True,'cube_view':'static'}) 

tf_env =  tf_py_environment.TFPyEnvironment( env )

In [17]:
actions, positions, theta_bias, rwds, t_rwd, dist_solid , vol = run_episode_test(env,tf_env,policy)
print(actions)
print(positions)
print(t_rwd)
print(dist_solid)
plot_vol_cam(vol,positions)

KeyError: 142

In [7]:
actions, positions, theta_bias, rwds, t_rwd, dist_solid , vol = run_rnd_episode_test(env,n_images)
print(actions)
print(positions)
print(t_rwd)
print(dist_solid)
plot_vol_cam(vol,positions)

[20, 102, 121, 74, 87, 116, 99, 103, 130]
[[106, 3], [110, 3], [143, 3], [3, 2], [26, 3], [54, 3], [91, 3], [124, 1], [157, 3], [19, 3]]
0.6578985492249426
0.6578985492249426


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
positions , rwd, vol  = run_uniform_test(models_path,train_models,n_images, 0)
print(positions)
print(rwd)

[[0, 0], [18, 0], [36, 0], [54, 0], [72, 0], [90, 0], [108, 0], [126, 0], [144, 0], [162, 0]]
0.4237346876207995


In [9]:
plot_vol_cam(vol,positions)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …